<a href="https://www.kaggle.com/code/prayas0/home-sensor-data-outlier-detection?scriptVersionId=227884721" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### This exercise is aimed at finding outliers in the ping, humidity or temperature of a given room. I would want to present my approach towards it and possibly discuss the pros and cons of doing any step

In [ ]:
import os
import plotly.express as px
import numpy as np 
import pandas as pd
import os
os.chdir('/kaggle/input/home-sensordata')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
def get_univariate_outliers(x, verbose = True, return_all = True):
    '''
    This function takes in column name and returns outliers based on boxplot formula and 3 sigma rule
    
    Args : 
        x : Pandas series or data frame feature
        
    Returns :
        Outlier flag on higher side, lower side and logical or of both flags
    '''
    
    mean, dev, q1, q3 = x.describe().loc[['mean','std','25%','75%']]

    iqr = q3 - q1
    lower_whisker = q1 - (1.5 * iqr)
    upper_whisker = q3 + (1.5 * iqr)

    sigma_3_pos = mean + (3 * dev)
    sigma_3_neg = mean - (3 * dev)

    

    out_low = x.apply(lambda x: 1 if (x < lower_whisker) | (x < sigma_3_neg) else 0)
    out_high = x.apply(lambda x: 1 if (x > upper_whisker) | (x > sigma_3_pos) else 0)
    out = x.apply(lambda x: 1 if (x < lower_whisker) | (x < sigma_3_neg) | (x > upper_whisker) | (x > sigma_3_pos) else 0)
#     pd.Series(out_low.astype(bool) | out_high.astype(bool)).astype(int)
    
    if verbose:
        print(f'lower whisker : {lower_whisker:.2f}, lower_3sigma : {sigma_3_neg:.2f} while min is: {x.min()}, upper_whisker : {upper_whisker:.2f} , upper_3sigma : {sigma_3_pos:.2f} while max is {x.max()}')
        print(f'Outliers on low side : {out_low.sum()} & on upper side : {out_high.sum()}')
    
    if return_all:
        return out_low, out_high, out
    else:
        return list(out)

In [ ]:
### Checking for missing values
df = pd.read_csv('data.csv')
df.isna().sum(axis=0)

### No missing values in data, time_id to be converted to datetime

In [ ]:
### checking all the features
df['time_id'] = pd.to_datetime(df['time_id'])
df['time_id'].describe()

#### The time column can be converted to Month, day (Sunday, Monday), weekday vs weekend, Date (might not prove very helpful as patterns are usually driven by day name as comapred to dates apart from financial aspects of a house), hour and minute

In [ ]:
df['mon_yr'] = (df['time_id'].dt.year.astype(str) + df['time_id'].dt.month.astype(str).str.zfill(2)).astype(int)
df['month'] = df['time_id'].dt.month_name()
df['date'] = df['time_id'].dt.day
df['day_type'] = df['time_id'].dt.day_name()
df['hour'] = df['time_id'].dt.hour
df['minute'] = df['time_id'].dt.minute

In [ ]:
df['ping_ms'].describe()

### Univariate outliers can be seen, ping is below 17ms in 75% of the collected data

In [ ]:
df['temperature_c'].describe()

### 54 degrees feels a bit extreme

In [ ]:
df['humidity_p'].describe()

In [ ]:
multiple_time_id = df['time_id'].value_counts().reset_index().query("time_id > 1")['index']
len(multiple_time_id)

#### value counts shows 31 time stamps with multiple records, we would investigate them

In [ ]:
df.loc[df['time_id'].isin(multiple_time_id)].sort_values(by = 'time_id').groupby('month').agg({'time_id':'count'})

* Given the data generation mechanism is of electronic nature, we may not want to remove/impute these duplicate data points as they are not collected due to errors.
* Many duplicates in October

In [ ]:
df.groupby('month').agg({'ping_ms':['mean','median'],'temperature_c':['mean','median'],'humidity_p':['mean','median'],'time_id':'count'}).style.format('{:.2f}').background_gradient("Blues",subset = ['ping_ms','temperature_c','humidity_p'])

#### Are we on to something 
* ping in September, October and January
* humidity in September, October (maybe due to low data count) or late monsoon?
* Difference between mean and median ping might be suggestive of outliers within the months as well

#### Outlier detection can be done using single feature at a time and also through multivariate analysis, we begin with univariate
## Pings

In [ ]:
px.box(df, y = 'ping_ms', x = 'month')

In [ ]:
df.groupby('month')['ping_ms'].describe().style.format('{:.2f}').background_gradient("Blues",subset = ['mean','std','min','25%','50%','75%','max'], axis = 'index')

#### Ping spikes in January, and as highlighted before, high pings in October and September, very high minimum and 25%

In [ ]:
help(get_univariate_outliers)

In [ ]:
### only using the overall outlier flag
_, _, out_ping = get_univariate_outliers(df["ping_ms"])

#### We can not have low outliers in ping but definitely there are spikes in ping which can be treated as outliers

In [ ]:
df['uni_out_ping'] = out_ping
df.groupby(['month','uni_out_ping']).agg({'time_id':'count','ping_ms':'mean'}).style.background_gradient("Reds", subset = ['ping_ms'])

## Temperature & Humidity

In [ ]:
df.groupby(['month'])['temperature_c'].describe()

In [ ]:
px.scatter(df, x = "temperature_c", y = "humidity_p")

* Max temperatures of 54 and 48 in Oct - Jan might be high/outlier
* We can see outlier points humidity and temperature in scatter plot
* Analysis at granularity of minute might not work for temperature and humidity, as they don't change much every minute
* We might have a very hot day or humid day but not a very hot minute/hour (adding unnecessary humour 😅 😅)

#### Let's create a change variable by shifting the temperature variable by one and subtract the lag variable and temperature, and if there are spikes in recording the temperature then the change variable will be symmetric around the spike 

In [ ]:
df['temp_lag'] = df['temperature_c'].shift(1)
df['temp_change'] = (df['temperature_c'] - df['temp_lag'])
df['abs_temp_change'] = df['temp_change'].abs()

df.sort_values(by = ['abs_temp_change','time_id'], ascending = [False,True]).head(6)

In [ ]:
temp_change = df['abs_temp_change'].value_counts().reset_index()
temp_change.columns = ['temp_change', 'record_count']
temp_change = temp_change.sort_values(by = ['temp_change'])
temp_change['temp_change'] = temp_change['temp_change'].astype(str)
temp_change['reverse_count'] = 1/temp_change['record_count']

px.bar(temp_change, y = "temp_change", x = "reverse_count", title = "unusual temperature change", orientation = "h", 
       labels = {"temp_change":"Temperature Change (C)", "reverse_count":"Inverse Frequency"}, hover_data = ['temp_change'])

In [ ]:
df['uni_out_temp'] = df['abs_temp_change'].apply(lambda x: 1 if x >= 5 else 0)
df['uni_out_temp'].sum()

In [ ]:
df['hum_lag'] = df['humidity_p'].shift(1)
df['hum_change'] = (df['humidity_p'] - df['hum_lag'])
df['abs_hum_change'] = df['hum_change'].abs()

df.sort_values(by = ['abs_hum_change','time_id'], ascending = [False,True]).head(6)

In [ ]:
hum_change = df['abs_hum_change'].value_counts().reset_index()
hum_change.columns = ['hum_change', 'record_count']
hum_change = hum_change.sort_values(by = ['hum_change'])
hum_change['hum_change'] = hum_change['hum_change'].astype(str)
hum_change['reverse_count'] = 1/hum_change['record_count']

px.bar(hum_change, y = "hum_change", x = "reverse_count", title = "unusual humidity change", orientation = "h", 
       labels = {"hum_change":"Humidity Change (Percentage)", "reverse_count":"Inverse Frequency"})

In [ ]:
df['uni_out_hum'] = df['abs_hum_change'].apply(lambda x: 1 if x >= 5 else 0)
df['uni_out_hum'].sum()

In [ ]:
df['uni_out'] = (df['uni_out_ping'] | df['uni_out_temp'] | df['uni_out_hum'])
df['uni_out'] = df['uni_out'].apply(lambda x: "Population" if x == 0 else "Outlier")

## Multivariate Outlier Detection

In [ ]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df[['ping_ms', 'temperature_c', 'humidity_p']])

## Isolation Forest

In [ ]:
iso = IsolationForest(random_state = 32, n_estimators = 100, bootstrap = True, contamination = 'auto')

df['iso'] = iso.fit_predict(scaled_df)
df['iso'] = df['iso'].apply(lambda x: "Population" if x == 1 else "Outlier")
df['iso'].value_counts()

In [ ]:
pd.crosstab(df['uni_out'],df['iso'])

In [ ]:
px.histogram(df, x = "ping_ms", color = "iso", marginal="box", title = "Outlier vs population (ping)")

In [ ]:
px.histogram(df, x = "temperature_c", color = "iso", marginal="box", title = "Outlier vs population (temperature)")

In [ ]:
px.histogram(df, x = "humidity_p", color = "iso", marginal="box", title = "Outlier vs population (humidity)")

## Local Outlier Factor

##### LOF might not be successful here, as we saw many pings in Januray at 2000 ms it might flag them as population blob which may not be correct because lof works using nearest neighbors

In [ ]:
lof = LocalOutlierFactor()

df['lof'] = lof.fit_predict(scaled_df)
df['lof'] = df['lof'].apply(lambda x: "Population" if x == 1 else "Outlier")
df['lof'].value_counts()

In [ ]:
px.histogram(df, x = "ping_ms", color = "lof", marginal="box", title = "Outlier vs population (ping)")

## Conclusion

In [ ]:
df.groupby(['iso','uni_out']).agg({'time_id':'count', 'temperature_c':['min', 'mean', 'max'], 'humidity_p':['min', 'mean', 'max'], 'ping_ms':['min', 'mean', 'max']}).style.format(precision = 2).background_gradient("Blues", subset = ['temperature_c', 'humidity_p', 'ping_ms'])

##### The "confusion matrix" between univariate and isolation forest can be further checked to understand more how isolation forest is working